# Assign category based on RSS feed or URL term

In [6]:
import pickle
from urllib.parse import urlparse, parse_qs

In [7]:
import os
import pymongo
import pandas as pd
mongoPass = os.environ['mongoPass']
# myclient = pymongo.MongoClient("mongodb+srv://axme100:{}@cluster0-5jopz.mongodb.net/test?retryWrites=true&w=majority".format(mongoPass))
myclient = pymongo.MongoClient()
mydb = myclient["finalProject"]
main = mydb["main_processed_article"]

# Get predefined URL TERMS

In [8]:
filename = 'url_terms'
infile = open(filename,'rb')
url_terms = pickle.load(infile)
infile.close()

# Function to assign terms

In [9]:
def get_category_from_url(url, publication):
    '''Input: URL and Publication
       Output: Discrete category
    '''
    
    # First create a dictionary that will check to see
    # If we can just get the category from the XML term
    xmlCategories = {'Expansión - Emprendedores': 'negocio',
                    'Expansión - Nacional': 'méxico',
                    'Expansión - Dinero': 'negocio',
                    'Expansión - Carrera': 'negocio',
                    'Expansión - Opinión': 'opinión',
                    'Expansión - Tecnología': 'tecnología',
                    'Expansión - Empresas': 'negocio'}
    
    # First check to see if the publication is in the dictionary
    # That we have created above, if it is then just return it
    if publication in xmlCategories:
        return xmlCategories[publication]
    
    # In case it is not, then we need to get the category from the URL
    else:
        parsed_url = urlparse(url)
        parsed = parsed_url.path
        potential_paths = parsed.split('/')
        
        for path in potential_paths:
            if path in url_terms:
                return url_terms[path]
            else:
                pass
        
        return "otro"
    

# Apply function to MONGO DB

In [10]:
cursor = main.find({})

In [11]:
# Create a new column called cateogry applying the function
# raw_article_data['category'] = raw_article_data.apply(lambda x: get_category_from_url(x.url, x.publication), axis=1)

In [13]:
for article in cursor:
    
    # First check if the article has already been classified
    myquery = { "_id": article['_id'] }
    
    article_category = list(main.find(myquery, {"_id": 0, "category": 1}))
    article_category = article_category[0]['category']
    
    # If it has not been classified yet then make a prediction and upate the corresponding field
    if not article_category:
    
        target_category = get_category_from_url(article['url'], article['publication'])
        
        # See: https://www.w3schools.com/python/python_mongodb_update.asp
        newvalues = { "$set": { "category": target_category } }
        main.update_one(myquery, newvalues)
    else:
        print("Article already has category")